# Import client

In [1]:
import pymongo
from dbclient import DBClient

In [2]:
db_client = DBClient()
db_client.check_connection()
db_client.collections.keys()

Pinged your deployment. You successfully connected to MongoDB!


dict_keys(['members', 'athletes', 'coaches', 'activities', 'session_types', 'access_types', 'logins'])

# Lookup tables

In [3]:
db_client.clear_all("access_types")

access_types_lookup = [
    {"name": "invalid", "description": "No access to any application data"},
    {"name": "admin", "description": "Full access to the application"},
    {"name": "individual", "description": "Access to the personal data only"},
    {"name": "reps_coach", "description": "Access to the REPS data only"},
]

db_client.collections["access_types"].insert_many(access_types_lookup)

# db_client.collections["access_types"].create_index([("accessId", pymongo.ASCENDING)], unique=True)

db_client.collections["access_types"].find_one({"name": "admin"})["_id"]

True
All entries successfully cleared from access_types table.


ObjectId('67140c2d443bb10f35ce2325')

## Member Declaration

In [4]:
group_ids = {
    group: db_client.collections["access_types"].find_one({"name": group})["_id"]
    for group in ["admin", "individual", "reps_coach", "invalid"]
}

member_data = [
    # System Admins 
    {"accessId": group_ids["admin"], "first_name": "Ben", "last_name": "Hoskings", "strava_id": 38807221, "dob": "27/07/2002"},
    
    # Coaches
    {"accessId": group_ids["reps_coach"], "first_name": "Clark", "last_name": "Roberts", "strava_id": 38781770, "dob": ""},
    
    # Standard Members
    {"accessId": group_ids["individual"], "first_name": "Neil", "last_name": "Hoskings", "strava_id": 963223, "dob": "14/08/1964"},
    {"accessId": group_ids["individual"], "first_name": "Becky", "last_name": "Hoskings", "strava_id": 98588012, "dob": "13/03/1972"},
    {"accessId": group_ids["individual"], "first_name": "Tom", "last_name": "Beesley", "strava_id": 33370063, "dob": ""},
    {"accessId": group_ids["individual"], "first_name": "Sophie", "last_name": "Hurst", "strava_id": 71665908, "dob": ""},
    {"accessId": group_ids["individual"], "first_name": "Wais", "last_name": "Yip", "strava_id": 14188705, "dob": ""}
]

In [5]:
db_client.clear_all("members")
db_client.collections["members"].insert_many(member_data)

True
All entries successfully cleared from members table.


InsertManyResult([ObjectId('67140c2d443bb10f35ce2328'), ObjectId('67140c2d443bb10f35ce2329'), ObjectId('67140c2d443bb10f35ce232a'), ObjectId('67140c2d443bb10f35ce232b'), ObjectId('67140c2d443bb10f35ce232c'), ObjectId('67140c2d443bb10f35ce232d'), ObjectId('67140c2d443bb10f35ce232e')], acknowledged=True)

In [10]:
group_ids = {
    group: db_client.collections["access_types"].find_one({"name": group})["_id"]
    for group in ["admin", "individual", "reps_coach", "invalid"]
}

login_details = [
    {"memberId": db_client.get_object_id("members", {"first_name": "Ben", "last_name": "Hoskings"}), 
     "username": "b_hoskings", "password": "bHos"},
    {"memberId": db_client.get_object_id("members", {"first_name": "Clark", "last_name": "Roberts"}), 
     "username": "c_roberts", "password": "cRob"}
]

In [11]:
db_client.clear_all("logins")
db_client.collections["logins"].insert_many(login_details)

True
All entries successfully cleared from logins table.


InsertManyResult([ObjectId('67140cb6443bb10f35ce2330'), ObjectId('67140cb6443bb10f35ce2331')], acknowledged=True)

## Coach Declaration

In [12]:
coach_data = [
    {"memberId": db_client.get_object_id("members", {"first_name": "Clark", "last_name": "Roberts"}), 
     "startDate": "2020-02-01", "lastTrainingDate": "2021-01-01"}
]

db_client.clear_all("coaches")
db_client.collections["coaches"].insert_many(coach_data)

True
All entries successfully cleared from coaches table.


InsertManyResult([ObjectId('67140d5b443bb10f35ce2332')], acknowledged=True)

## Athlete Declaration

In [15]:
coach_ids = {
    coach_name[0]: db_client.get_object_id(
        "members", 
        db_client.get_object_id("members", {"first_name": coach_name[0], "last_name": coach_name[1]})
    )
    for coach_name in [
        ("Clark", "Roberts")
    ]
}

athlete_data = [
    {"memberId": db_client.get_object_id("members", {"first_name": "Clark", "last_name": "Roberts"}), 
     "coachId": None}, 
    {"memberId": db_client.get_object_id("members", {"first_name": "Ben", "last_name": "Hoskings"}), 
     "coachId": coach_ids["Clark"]}, 
    {"memberId": db_client.get_object_id("members", {"first_name": "Sophie", "last_name": "Hurst"}), 
     "coachId": coach_ids["Clark"]}, 
    {"memberId": db_client.get_object_id("members", {"first_name": "Wais", "last_name": "Yip"}), 
     "coachId": coach_ids["Clark"]}
]
db_client.clear_all("athletes")
db_client.collections["athletes"].insert_many(athlete_data)

True
All entries successfully cleared from athletes table.


InsertManyResult([ObjectId('67140fee443bb10f35ce2335'), ObjectId('67140fee443bb10f35ce2336'), ObjectId('67140fee443bb10f35ce2337'), ObjectId('67140fee443bb10f35ce2338')], acknowledged=True)